<a href="https://colab.research.google.com/github/qazwsxedc900/ML-2021-Spring/blob/main/%E3%80%8CSHARE_MLSpring2021_HW2_1_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2-1 Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs


## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task,
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
From (redirected): https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR&confirm=t&uuid=e1f74f79-6192-4b0f-b888-f90d2837a2df
To: /content/data.zip
100% 372M/372M [00:03<00:00, 99.9MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            #y = y.astype(np.int)
            y = y.astype(int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [4]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [5]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [6]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

30

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [7]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 128)
        self.out = nn.Linear(128, 39)

        self.act_fn = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.act_fn(x)

        x = self.layer3(x)
        x = self.act_fn(x)

        x = self.out(x)

        return x

## Training

In [8]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [9]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [10]:
'''
# 安裝 torch_xla 函式庫
# 移除特定版本的 wheel 檔案，讓 pip 根據環境自動選擇合適的版本
!pip install torch_xla

import torch_xla.core.xla_model as xm # 引入 torch_xla

# 檢查設備，現在包含對 TPU 的偵測
def get_device():
  # 使用 torch_xla.core.xla_model.xla_device() 來取得 TPU 設備
  return xm.xla_device()

device = get_device()
print(f'DEVICE: {device}')

# 固定隨機種子
def same_seeds(seed):
    torch.manual_seed(seed)
    # 對於 TPU，使用 xm.set_rng_state 設置隨機種子
    if device.type == 'xla':
        xm.set_rng_state(seed)
    elif torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# 固定隨機種子以確保可重複性
same_seeds(0)

# 訓練參數
num_epoch = 20               # 訓練的 epoch 數量
learning_rate = 0.0001       # 學習率

# 儲存模型檢查點的路徑
model_path = './model.ckpt'

# 建立模型，定義損失函數和優化器
model = Classifier().to(device) # 將模型移動到偵測到的設備 (TPU 或 CPU)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
'''

"\n# 安裝 torch_xla 函式庫\n# 移除特定版本的 wheel 檔案，讓 pip 根據環境自動選擇合適的版本\n!pip install torch_xla\n\nimport torch_xla.core.xla_model as xm # 引入 torch_xla\n\n# 檢查設備，現在包含對 TPU 的偵測\ndef get_device():\n  # 使用 torch_xla.core.xla_model.xla_device() 來取得 TPU 設備\n  return xm.xla_device()\n\ndevice = get_device()\nprint(f'DEVICE: {device}')\n\n# 固定隨機種子\ndef same_seeds(seed):\n    torch.manual_seed(seed)\n    # 對於 TPU，使用 xm.set_rng_state 設置隨機種子\n    if device.type == 'xla':\n        xm.set_rng_state(seed)\n    elif torch.cuda.is_available():\n        torch.cuda.manual_seed(seed)\n        torch.cuda.manual_seed_all(seed)\n    np.random.seed(seed)\n    torch.backends.cudnn.benchmark = False\n    torch.backends.cudnn.deterministic = True\n\n# 固定隨機種子以確保可重複性\nsame_seeds(0)\n\n# 訓練參數\nnum_epoch = 20               # 訓練的 epoch 數量\nlearning_rate = 0.0001       # 學習率\n\n# 儲存模型檢查點的路徑\nmodel_path = './model.ckpt'\n\n# 建立模型，定義損失函數和優化器\nmodel = Classifier().to(device) # 將模型移動到偵測到的設備 (TPU 或 CPU)\ncriterion = nn.CrossEntrop

In [11]:
'''
#start training
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0 # Initialize as float
    val_acc = 0.0
    val_loss = 0.0 # Initialize as float

    # 訓練階段
    model.train() # 將模型設定為訓練模式
    total_train_loss = 0.0 # 使用單獨的變數來累計純量損失
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) # 將資料移動到設備
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # 取得具有最高機率的類別索引
        batch_loss.backward()

        # 將累積損失和準確率的程式碼移到 xm.optimizer_step 之前
        total_train_loss += batch_loss.item()
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()

        # 在使用 TPU 時，使用 xm.optimizer_step 來執行優化器的 step
        xm.optimizer_step(optimizer)


    # 在迴圈結束後計算平均訓練損失
    train_loss = total_train_loss / len(train_loader)


    # 驗證階段
    if len(val_set) > 0:
        model.eval() # 將模型設定為評估模式
        total_val_loss = 0.0 # 使用單獨的變數來累計純量損失
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device) # 將資料移動到設備
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)

                # 累積純量損失和準確率
                total_val_loss += batch_loss.item()
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # 取得具有最高機率的類別索引


            # 在迴圈結束後計算平均驗證損失
            val_loss = total_val_loss / len(val_loader)

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss, val_acc/len(val_set), val_loss
            ))

            # 如果模型有改進，儲存當前的檢查點
            # 使用 TPU 時，建議使用 xm.save 來儲存模型狀態
            if val_acc > best_acc:
                best_acc = val_acc
                # torch.save(model.state_dict(), model_path) # 標準的儲存方式可能可以，但建議使用 xm.save
                xm.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss
        ))

# 如果沒有進行驗證，儲存最後一個 epoch 的模型
if len(val_set) == 0:
    # torch.save(model.state_dict(), model_path) # 標準的儲存方式可能可以，但建議使用 xm.save
    xm.save(model.state_dict(), model_path)
    print('saving model at last epoch')
'''

"\n#start training\nbest_acc = 0.0\nfor epoch in range(num_epoch):\n    train_acc = 0.0\n    train_loss = 0.0 # Initialize as float\n    val_acc = 0.0\n    val_loss = 0.0 # Initialize as float\n\n    # 訓練階段\n    model.train() # 將模型設定為訓練模式\n    total_train_loss = 0.0 # 使用單獨的變數來累計純量損失\n    for i, data in enumerate(train_loader):\n        inputs, labels = data\n        inputs, labels = inputs.to(device), labels.to(device) # 將資料移動到設備\n        optimizer.zero_grad()\n        outputs = model(inputs)\n        batch_loss = criterion(outputs, labels)\n        _, train_pred = torch.max(outputs, 1) # 取得具有最高機率的類別索引\n        batch_loss.backward()\n\n        # 將累積損失和準確率的程式碼移到 xm.optimizer_step 之前\n        total_train_loss += batch_loss.item()\n        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()\n\n        # 在使用 TPU 時，使用 xm.optimizer_step 來執行優化器的 step\n        xm.optimizer_step(optimizer)\n\n\n    # 在迴圈結束後計算平均訓練損失\n    train_loss = total_train_loss / len(train_loader)\n\n\n    # 驗證階段\

In [12]:

# fix random seed for reproducibility
same_seeds(0)

# get device
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


DEVICE: cuda


In [13]:

# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels)
                _, val_pred = torch.max(outputs, 1)

                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/020] Train Acc: 0.467302 Loss: 1.811661 | Val Acc: 0.567428 loss: 1.433065
saving model with acc 0.567
[002/020] Train Acc: 0.594383 Loss: 1.330665 | Val Acc: 0.628639 loss: 1.211098
saving model with acc 0.629
[003/020] Train Acc: 0.644506 Loss: 1.154064 | Val Acc: 0.660421 loss: 1.101215
saving model with acc 0.660
[004/020] Train Acc: 0.672216 Loss: 1.052246 | Val Acc: 0.676300 loss: 1.038718
saving model with acc 0.676
[005/020] Train Acc: 0.691347 Loss: 0.983103 | Val Acc: 0.685154 loss: 1.001852
saving model with acc 0.685
[006/020] Train Acc: 0.705615 Loss: 0.931955 | Val Acc: 0.689301 loss: 0.984177
saving model with acc 0.689
[007/020] Train Acc: 0.716344 Loss: 0.891687 | Val Acc: 0.694516 loss: 0.964627
saving model with acc 0.695
[008/020] Train Acc: 0.725881 Loss: 0.857907 | Val Acc: 0.697720 loss: 0.951889
saving model with acc 0.698
[009/020] Train Acc: 0.733717 Loss: 0.829495 | Val Acc: 0.696687 loss: 0.949866
[010/020] Train Acc: 0.741151 Loss: 0.803701 | Val Acc: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [14]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [15]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [16]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))